<a href="https://colab.research.google.com/github/harnalashok/LLMs/blob/main/agents_tutorial_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Refer YouTube Video
#  https://www.youtube.com/watch?v=bHn4dLJYIqE

## Roadmap

Part-4 This video focus:

- Function Calling Agents + Agent Runner
- Agentic RAG
- Project - ReACT Agent: Building a Search Assistant Agent


In [ ]:
!pip install llama-index

In [ ]:
!pip install duckduckgo-search

In [ ]:
!pip install llama-index-llms-mistralai llama-index-embeddings-mistralai

## Function Calling

In [ ]:
from llama_index.core.tools import FunctionTool
from duckduckgo_search import DDGS
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding # Agentic RAG (Function calling - not required)

import nest_asyncio
nest_asyncio.apply()

In [ ]:
def search(query:str) -> str:
  """
  Args:
      query: user prompt
  return:
  context (str): search results to the user query
  """
  # def search(query:str)
  req = DDGS()
  response = req.text(query,max_results=4)
  context = ""
  for result in response:
    context += result['body']
  return context

In [ ]:
search_tool = FunctionTool.from_defaults(fn=search)

In [ ]:
query = "Who was the Man of the series in India vs Bangalesh 2024 Test Match?"

In [ ]:
mistral_api = ""

In [ ]:
llm = MistralAI(model="mistral-large-latest",api_key=mistral_api)

In [ ]:
embed_model = MistralAIEmbedding(model_name="mistral-embed", api_key=mistral_api)

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
llm.complete("tell me about AI?").text

In [ ]:
function_call = llm.predict_and_call(
    [search_tool],
    user_msg = query,
    allow_parallel_tool_calls=True,
)

In [ ]:
type(function_call)

llama_index.core.chat_engine.types.AgentChatResponse

In [ ]:
function_call.response

"IND vs BDESH 2024/25, India vs Bangladesh Match Report: Jaiswal scored twin fifties as India registered a remarkable win despite two days of no play due to rain in Kanpur Matches ( 6 ) Women's T20 ...India won the two-match IND vs BAN test series by 2-0, with R Ashwin bagging the Man of the Series award. The Indian cricket team extended their home dominance by whitewashing Bangladesh in the two-match IND vs BAN test series. India won by the first test in Chennai by 280 runs and the second test in Kanpur by seven wickets.India defeated Bangladesh by seven wickets in the second Test at Kanpur, sealing the series 2-0. Yashasvi Jaiswal's 51 guided India to a successful chase of 95 runs. Follow Us Trending Photos IND vs BAN: Indian opener Yashavi Jaiswal's exceptional fifty in the second innings guided the hosts to a ...IND vs BDESH 2024/25, India vs Bangladesh Match Report: Bangladesh captain Najmul Hossain Shanto's 82 kept the bowling at bay, but Ashwin and Ravindra Jadeja shared nine wi

## Second Approach for Function Calling Agent

- Function Calling Agent Worker

In [ ]:
from llama_index.core.agent import FunctionCallingAgent, FunctionCallingAgentWorker

In [ ]:
agent = FunctionCallingAgent.from_tools(
    [search_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=True,
)

In [ ]:
response = agent.chat(query)

> Running step 0aae73a7-08d2-477a-9703-bb0d517d075a. Step input: Who was the Man of the series in India vs Bangalesh 2024 Test Match?
Added user message to memory: Who was the Man of the series in India vs Bangalesh 2024 Test Match?
=== Calling Function ===
Calling function: search with args: {"query": "Who was the Man of the series in India vs Bangalesh 2024 Test Match?"}
=== Function Output ===
IND vs BDESH 2024/25, India vs Bangladesh Match Report: Jaiswal scored twin fifties as India registered a remarkable win despite two days of no play due to rain in Kanpur Matches ( 6 ) Women's T20 ...IND vs BAN 2024, 2ND Test: Match Presentation Share video on : 1st Oct, 2024 14:17 mins 115.2 k IND vs BAN 2024, 2ND Test: India 2nd Innings Fall of Wickets ... Dressing Room BTS | Impact Fielder of the Series | India vs Bangladesh Test Series 2024 Share video on : 1st Oct, 2024 02:35 mins 98.4 k Rohit Sharma said we are going to go hammer and ...India vs Bangladesh, 2nd Test Day 5 ... Bangladesh 

In [ ]:
print(response)

The Man of the Series in the India vs Bangladesh 2024 Test Match was not explicitly mentioned in the search results. However, notable performances included Jaiswal scoring twin fifties and Ashwin and Ravindra Jadeja sharing nine wickets. For the most accurate and up-to-date information, please refer to official cricket sources.


## Approach -3 and the recommended one: Agent Runner

In [ ]:
from llama_index.core.agent import AgentRunner

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [search_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=True,
)
agent_runner = AgentRunner(agent_worker)

In [ ]:
runner_response = agent_runner.query(query)

Added user message to memory: Who was the Man of the series in India vs Bangalesh 2024 Test Match?
=== Calling Function ===
Calling function: search with args: {"query": "Who was the Man of the series in India vs Bangalesh 2024 Test Match?"}
=== Function Output ===
India vs Bangladesh 2024 T20I Squads. India T20I Squad vs Bangladesh: Suryakumar Yadav (captain), Abhishek Sharma, Sanju Samson (wicketkeeper), Rinku Singh, Hardik Pandya, Riyan Parag, Nitish Kumar Reddy, Shivam Dube, Washington Sundar, Ravi Bishnoi, Varun Chakaravarthy, Jitesh Sharma (wicketkeeper), Arshdeep Singh, Harshit Rana, Mayank Yadav. ...India won the two-match IND vs BAN test series by 2-0, with R Ashwin bagging the Man of the Series award. The Indian cricket team extended their home dominance by whitewashing Bangladesh in the two-match IND vs BAN test series.. India won by the first test in Chennai by 280 runs and the second test in Kanpur by seven wickets.IND vs BDESH 2024/25, India vs Bangladesh Match Report: Ja

In [ ]:
runner_response.response

'R Ashwin was the Man of the Series in the India vs Bangladesh 2024 Test Match. India won the two-match series by 2-0, with R Ashwin bagging the Man of the Series award. The Indian cricket team extended their home dominance by whitewashing Bangladesh in the two-match series. India won the first test in Chennai by 280 runs and the second test in Kanpur by seven wickets.'

In [ ]:
runner_chat = agent_runner.chat(query)

Added user message to memory: Who was the Man of the series in India vs Bangalesh 2024 Test Match?
=== Calling Function ===
Calling function: search with args: {"query": "Who was the Man of the series in India vs Bangalesh 2024 Test Match?"}
=== Function Output ===
IND vs BDESH 2024/25, India vs Bangladesh Match Report: Jaiswal scored twin fifties as India registered a remarkable win despite two days of no play due to rain in Kanpur Matches ( 6 ) Women's T20 ...Indian players celebrate with the trophy after winning the two match test series against Bangladesh in Kanpur, India, Tuesday, October 1, 2024. ... Sharma celebrates the wicket of Bangladesh's Mehidy Hasan Miraz on the fifth and final day of the second cricket test match between Bangladesh and India in Kanpur, India, Tuesday, October 1, 2024 ...IND vs BAN 2024, 2ND Test: Match Presentation Share video on : 1st Oct, 2024 14:17 mins 115.2 k IND vs BAN 2024, 2ND Test: India 2nd Innings Fall of Wickets ... Dressing Room BTS | Impact

In [ ]:
runner_chat.response

'The Man of the series in India vs Bangladesh 2024 Test Match was Rohit Sharma.'

## Agentic RAG

In [ ]:
from llama_index.core import SimpleDirectoryReader # load your documents

In [ ]:
documents = SimpleDirectoryReader(input_files=["/content/2409.16160v1.pdf"]).load_data()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [ ]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [ ]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

In [ ]:
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "This Agent is useful to summarize the Research paper in a simplied way by giving intuitions"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "This Agent is useful to answer to the user queries within the paper and retrieve the relevant piece of information"
    ),
)

In [ ]:
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [ ]:
response = query_engine.query("give me technical architecture details in the paper")

Selecting query engine 1: This Agent is useful to answer to the user queries within the paper and retrieve the relevant piece of information.


In [ ]:
print(response)

The provided context does not include explicit details about the technical architecture of the paper. It primarily discusses the capabilities and testing scenarios of a method related to character animation and control. For technical architecture details, you would need to refer to other sections of the paper that specifically describe the model, algorithms, or system design.


In [ ]:
#response.source_nodes

In [ ]:
response2 = query_engine.query("summarize the methodology from the paper")

Selecting query engine 0: This choice is most relevant because it mentions summarizing the research paper in a simplified way by giving intuitions, which aligns with the need to summarize the methodology from the paper..


In [ ]:
print(response2)

The methodology presented in the paper involves a novel framework called MIMO for controllable character video synthesis. The framework decomposes a 2D video into three spatial components: the main human, the underlying scene, and floating occlusions, using hierarchical layers based on 3D depth. These components are then encoded into canonical identity code, structured motion code, and full scene code, which serve as control signals for the synthesis process. The structured motion code is derived from a deformable human body model (SMPL) and camera parameters, enabling more dense pose representation with 3D occlusions. The canonical identity code is obtained by transforming the posed human image to a canonical result in standard A-pose using a pretrained human repose model. The scene and occlusion components are embedded with a shared VAE encoder and re-organized as a full scene code. These latent codes are inserted into a diffusion-based decoder as conditions for video reconstruction.

## ReACT Agent

Build a simple project: Search Assistant Agent

In [ ]:
from llama_index.core.agent import ReActAgent

In [ ]:
agent = ReActAgent.from_tools([search_tool], llm=llm, verbose=True,allow_parallel_tool_calls=True)

In [ ]:
response = agent.chat("tell me about AI with Tarun. Also do analysis and say should people follow him?")

> Running step 4e7cee18-bfa1-4d4e-ab07-d3e7406eb35b. Step input: tell me about AI with Tarun. Also do analysis and say should people follow him?
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: search
Action Input: {'query': 'AI with Tarun'}
Observation: 🚀 "Learn AI for FREE with Tarun!" 🚀Hold on tight, techies! You're about to launch into the world of AI with Tarun Jain - no money, just fun! 💸🧠By day, he...Tarun is instrumental in creating and developing IBM's next-generation AI platform, watsonx, a revolutionary leap forward in the world of AI for businesses. He presented at Think this week and shared an insightful session about the transformative potential of generative AI and foundation models and the role of watsonx in enabling such advancements.AI With Tarun | AI Engineer. I help you build and learn AI for free 👨‍💻 🗣️ AI/ML Interview Series 📰 Cr... YouTube. Instagram. X. Here is the Prompt Guide. AI With Tarun. Ins

In [ ]:
print(response)

Tarun Jain is an AI engineer who offers free resources to help people learn and build AI. He has expertise in generative AI, machine learning, image processing, and deep learning. Tarun works as a Developer Relations & Data Scientist at AI Planet and is a Google Developer Expert in AI/ML. He has also participated in Google Summer of Code 2023 (GSoC23) at caMicroscope and was an Ex-Research Intern at CCCIR. Additionally, he is involved in creating and developing IBM's next-generation AI platform, watsonx.

Analysis:
- **Expertise**: Tarun has a strong background in AI and machine learning, with experience in both industry and research.
- **Free Resources**: He provides free educational content, which can be very beneficial for those looking to learn AI without financial barriers.
- **Industry Involvement**: His work with IBM and other organizations indicates that he is well-connected and up-to-date with the latest advancements in the field.
- **Community Engagement**: Tarun is active on